In [34]:
import sqlite3
import pandas as pd

In [35]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [36]:
data = pd.read_csv('https://raw.githubusercontent.com/dsreload/sql_2/master/boston.csv')
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [37]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

**1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)**

Первый вариант (по столбцам):

In [38]:
pd.read_sql(
    """
    SELECT SUM(CASE WHEN CRIM IS NULL THEN 1 ELSE 0 END) as cnt_null_CRIM,
           SUM(CASE WHEN ZN IS NULL THEN 1 ELSE 0 END) as cnt_null_ZN,
           SUM(CASE WHEN INDUS IS NULL THEN 1 ELSE 0 END) as cnt_null_INDUS,
           SUM(CASE WHEN CHAS IS NULL THEN 1 ELSE 0 END) as cnt_null_CHAS,
           SUM(CASE WHEN NOX IS NULL THEN 1 ELSE 0 END) as cnt_null_NOX
    FROM boston
    """,
    con,
)

,cnt_null_CRIM,cnt_null_ZN,cnt_null_INDUS,cnt_null_CHAS,cnt_null_NOX
0,0,0,0,0,0


Второй вариант (по строкам):

In [39]:
pd.read_sql(
    """
    SELECT 'CRIM' as column,
           SUM(CASE WHEN CRIM IS NULL THEN 1 ELSE 0 END) as cnt_nulls
    FROM boston
    UNION
    SELECT 'ZN' as column,
           SUM(CASE WHEN ZN IS NULL THEN 1 ELSE 0 END) as cnt_nulls
    FROM boston
    UNION
    SELECT 'INDUS' as column,
           SUM(CASE WHEN INDUS IS NULL THEN 1 ELSE 0 END) as cnt_nulls
    FROM boston
    UNION
    SELECT 'CHAS' as column,
           SUM(CASE WHEN CHAS IS NULL THEN 1 ELSE 0 END) as cnt_nulls
    FROM boston
    UNION
    SELECT 'NOX' as column,
           SUM(CASE WHEN NOX IS NULL THEN 1 ELSE 0 END) as cnt_nulls
    FROM boston
    """,
    con,
)

,column,cnt_nulls
0,CHAS,0
1,CRIM,0
2,INDUS,0
3,NOX,0
4,ZN,0


**2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)**

In [40]:
pd.read_sql(
    """
    SELECT 'CRIM' as column,
           COUNT(DISTINCT CRIM) as cnt_distinct_values
    FROM boston
    UNION
    SELECT 'ZN' as column,
           COUNT(DISTINCT ZN) as cnt_distinct_values
    FROM boston
    UNION
    SELECT 'INDUS' as column,
           COUNT(DISTINCT INDUS) as cnt_distinct_values
    FROM boston
    UNION
    SELECT 'CHAS' as column,
           COUNT(DISTINCT CHAS) as cnt_distinct_values
    FROM boston
    UNION
    SELECT 'NOX' as column,
           COUNT(DISTINCT NOX) as cnt_distinct_values
    FROM boston
    """,
    con,
)

,column,cnt_distinct_values
0,CHAS,2
1,CRIM,504
2,INDUS,76
3,NOX,81
4,ZN,26


**3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках**

Реализуем запрос, возращающий данные вида (нахвание колонки, медиана, минимальное значение):

In [41]:
pd.read_sql(
    """
    SELECT 'CRIM' as ColName,
           AVG(value) as Median,
           (SELECT MIN(CRIM) FROM boston) as MinValue
    FROM (SELECT CRIM as value
          FROM boston
          ORDER BY CRIM
          LIMIT 2 - (SELECT COUNT(CRIM) FROM boston) % 2
          OFFSET (SELECT (COUNT(CRIM) - 1) / 2 FROM boston)
    )    
    """,
    con,
)

,ColName,Median,MinValue
0,CRIM,0.25651,0.00632


Затем через UNION соединим результаты для всех колонок, а затем выберем те, у которых медиана и минимальное значение совпадают:

In [42]:
pd.read_sql(
    """
    SELECT ColName
    FROM (
            SELECT 'CRIM' as ColName,
                AVG(value) as Median,
                (SELECT MIN(CRIM) FROM boston) as MinValue
            FROM (SELECT CRIM as value
                FROM boston
                ORDER BY CRIM
                LIMIT 2 - (SELECT COUNT(CRIM) FROM boston) % 2
                OFFSET (SELECT (COUNT(CRIM) - 1) / 2 FROM boston)
            )
            UNION
            SELECT 'ZN' as ColName,
                AVG(value) as Median,
                (SELECT MIN(ZN) FROM boston) as MinValue
            FROM (SELECT ZN as value
                FROM boston
                ORDER BY ZN
                LIMIT 2 - (SELECT COUNT(ZN) FROM boston) % 2
                OFFSET (SELECT (COUNT(ZN) - 1) / 2 FROM boston)
            )
            UNION
            SELECT 'INDUS' as ColName,
                AVG(value) as Median,
                (SELECT MIN(INDUS) FROM boston) as MinValue
            FROM (SELECT INDUS as value
                FROM boston
                ORDER BY INDUS
                LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                OFFSET (SELECT (COUNT(INDUS) - 1) / 2 FROM boston)
            )
            UNION
            SELECT 'CHAS' as ColName,
                AVG(value) as Median,
                (SELECT MIN(CHAS) FROM boston) as MinValue
            FROM (SELECT CHAS as value
                FROM boston
                ORDER BY CHAS
                LIMIT 2 - (SELECT COUNT(CHAS) FROM boston) % 2
                OFFSET (SELECT (COUNT(CHAS) - 1) / 2 FROM boston)
            )
            UNION
            SELECT 'NOX' as ColName,
                AVG(value) as Median,
                (SELECT MIN(NOX) FROM boston) as MinValue
            FROM (SELECT NOX as value
                FROM boston
                ORDER BY NOX
                LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                OFFSET (SELECT (COUNT(NOX) - 1) / 2 FROM boston)
            )
    )
    WHERE Median =  MinValue
    """,
    con,
)

,ColName
0,CHAS
1,ZN


С учётом того, что количество уникальных элементов в этих колонках не равно 1 (см. задание 2), тогда можно сделать вывод, что кол-во элементов, равных минимальному значению, составляет более половины числа всех элементов данной колонки.

**4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.**

Для начала выберем дома с максимальной стоимостью и посчитаем их среднее кол-во комнат:

In [43]:
pd.read_sql(
    """
    SELECT AVG(RM) as AvgRM
    FROM boston
    WHERE MEDV = (SELECT MAX(MEDV) 
                  FROM boston
                  ORDER BY MEDV DESC)
    """,
    con,
)

,AvgRM
0,7.484


Объединим вышеупомянутый запрос с запросом на нахождение среднее кол-ва комнат в самых дешёвых домах:

In [44]:
pd.read_sql(
    """
    SELECT 25 as NumCheapestHouses,
           AVG(RM) as AvgRM_MostCheapest,
           (SELECT AVG(RM) as AvgRM
            FROM boston
            WHERE MEDV = (SELECT MAX(MEDV) 
                          FROM boston
                          ORDER BY MEDV DESC)
                          ) as AvgRM_ModtExpensive
    FROM (SELECT RM
          FROM boston
          ORDER BY MEDV
          LIMIT 25
    )
    """,
    con,
)

,NumCheapestHouses,AvgRM_MostCheapest,AvgRM_ModtExpensive
0,25,5.74784,7.484


Тогда результирующий запрос:

In [45]:
pd.read_sql(
    """
    WITH avg_most_exp as (
    SELECT AVG(RM) as AvgRM
    FROM boston
    WHERE MEDV = (SELECT MAX(MEDV) 
                  FROM boston
                  ORDER BY MEDV DESC)
    )

    SELECT *,
           AvgRM_MostExpensive - AvgRM_MostCheapest as Difference
    FROM (
            SELECT 25 as NumCheapestHouses,
                   AVG(RM) as AvgRM_MostCheapest,
                   (SELECT * FROM avg_most_exp) as AvgRM_MostExpensive
            FROM (SELECT RM
                  FROM boston
                  ORDER BY MEDV
                  LIMIT 25
            )
            UNION
            SELECT 50 as NumCheapestHouses,
                   AVG(RM) as AvgRM_MostCheapest,
                   (SELECT * FROM avg_most_exp) as AvgRM_MostExpensive
            FROM (SELECT RM
                  FROM boston
                  ORDER BY MEDV
                  LIMIT 50
            )
            UNION
            SELECT 100 as NumCheapestHouses,
                   AVG(RM) as AvgRM_MostCheapest,
                   (SELECT * FROM avg_most_exp) as AvgRM_MostExpensive
            FROM (SELECT RM
                  FROM boston
                  ORDER BY MEDV
                  LIMIT 100
            )
            UNION
            SELECT 200 as NumCheapestHouses,
                   AVG(RM) as AvgRM_MostCheapest,
                   (SELECT * FROM avg_most_exp) as AvgRM_MostExpensive
            FROM (SELECT RM
                  FROM boston
                  ORDER BY MEDV
                  LIMIT 200
            )
            UNION
            SELECT 300 as NumCheapestHouses,
                   AVG(RM) as AvgRM_MostCheapest,
                   (SELECT * FROM avg_most_exp) as AvgRM_MostExpensive
            FROM (SELECT RM
                  FROM boston
                  ORDER BY MEDV
                  LIMIT 300
            )
        )
    """,
    con,
)

,NumCheapestHouses,AvgRM_MostCheapest,AvgRM_MostExpensive,Difference
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


Видно, чем стоимость дома становится больше, тем и среднее кол-во комнат в нём повышается, но не линейно.

**5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех знаечений LSTAT. Напишите какой вывод можно сделать по этим данным.**

In [46]:
pd.read_sql(
    """
    SELECT LSTAT, MEDV, RankInTotal
    FROM (SELECT *, row_number() OVER(ORDER BY LSTAT) as RankInTotal
          FROM boston)
    WHERE MEDV = (SELECT MAX(MEDV)
                  FROM boston
                  ORDER BY MEDV DESC)
    """,
    con,
)

,LSTAT,MEDV,RankInTotal
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,16
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


Как видим, среди самых дорогих домов сначала при увеличении LSTAT ранг повышется не сильно, а затем уже где-то с 10 дома начинается резкий рост.

**6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)**

In [47]:
pd.read_sql(
    """
    SELECT 'Граничит' as bounds_with_river,
           AVG(MEDV) as mean_price
    FROM boston
    WHERE CHAS = 1
    UNION
    SELECT 'Не граничит' as bounds_with_river,
           AVG(MEDV) as mean_price
    FROM boston
    WHERE CHAS = 0
    """,
    con,
)

,bounds_with_river,mean_price
0,Граничит,28.440000
1,Не граничит,22.093843


Или возможен более лаконичный вариант через оконную функцию:

In [48]:
pd.read_sql(
    """
    SELECT DISTINCT CHAS as bounds_with_river,
           AVG(MEDV) OVER(PARTITION BY CHAS) as mean_price
    FROM boston
    """,
    con,
)

,bounds_with_river,mean_price
0,0.0,22.093843
1,1.0,28.440000


**7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.**

In [49]:
pd.read_sql(
    """
    SELECT ColNAme
    FROM (
        SELECT 'CRIM' as ColName,
               (SELECT AVG(CRIM) FROM boston WHERE CHAS = 1) as AvgValueWithRiver,
               (SELECT AVG(CRIM) FROM boston WHERE CHAS = 0) as AvgValueWithoutRiver
        UNION
        SELECT 'ZN' as ColName,
               (SELECT AVG(ZN) FROM boston WHERE CHAS = 1) as AvgValueWithRiver,
               (SELECT AVG(ZN) FROM boston WHERE CHAS = 0) as AvgValueWithoutRiver
        UNION
        SELECT 'INDUS' as ColName,
               (SELECT AVG(INDUS) FROM boston WHERE CHAS = 1) as AvgValueWithRiver,
               (SELECT AVG(INDUS) FROM boston WHERE CHAS = 0) as AvgValueWithoutRiver
        UNION
        SELECT 'NOX' as ColName,
               (SELECT AVG(NOX) FROM boston WHERE CHAS = 1) as AvgValueWithRiver,
               (SELECT AVG(NOX) FROM boston WHERE CHAS = 0) as AvgValueWithoutRiver
    )
    WHERE AvgValueWithRiver > AvgValueWithoutRiver
    """,
    con,
)

,ColName
0,INDUS
1,NOX


В целом, ожидаемый результат, так как промышленная застройка часто находится у воды, и соответственно выбросы оксидов азота также будут выше там, где есть промышленность и река.

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [50]:
pd.read_sql(
    """
    SELECT 10 as Percentile,
           (SELECT AVG(value)
           FROM (SELECT INDUS as value
                 FROM boston
                 ORDER BY INDUS
                 LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                 OFFSET (SELECT (COUNT(INDUS) - 1) * 1 / 10 FROM BOSTON)
                 )) as value_INDUS,
           (SELECT AVG(value)
           FROM (SELECT NOX as value
                 FROM boston
                 ORDER BY NOX
                 LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                 OFFSET (SELECT (COUNT(NOX) - 1) * 1 / 10 FROM BOSTON)
                 )) as value_NOX
    UNION
    SELECT 20 as Percentile,
           (SELECT AVG(value)
           FROM (SELECT INDUS as value
                 FROM boston
                 ORDER BY INDUS
                 LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                 OFFSET (SELECT (COUNT(INDUS) - 1) * 2 / 10 FROM BOSTON)
                 )) as value_INDUS,
           (SELECT AVG(value)
           FROM (SELECT NOX as value
                 FROM boston
                 ORDER BY NOX
                 LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                 OFFSET (SELECT (COUNT(NOX) - 1) * 2 / 10 FROM BOSTON)
                 )) as value_NOX
    UNION
    SELECT 30 as Percentile,
           (SELECT AVG(value)
           FROM (SELECT INDUS as value
                 FROM boston
                 ORDER BY INDUS
                 LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                 OFFSET (SELECT (COUNT(INDUS) - 1) * 3 / 10 FROM BOSTON)
                 )) as value_INDUS,
           (SELECT AVG(value)
           FROM (SELECT NOX as value
                 FROM boston
                 ORDER BY NOX
                 LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                 OFFSET (SELECT (COUNT(NOX) - 1) * 3 / 10 FROM BOSTON)
                 )) as value_NOX
    UNION
    SELECT 40 as Percentile,
           (SELECT AVG(value)
           FROM (SELECT INDUS as value
                 FROM boston
                 ORDER BY INDUS
                 LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                 OFFSET (SELECT (COUNT(INDUS) - 1) * 4 / 10 FROM BOSTON)
                 )) as value_INDUS,
           (SELECT AVG(value)
           FROM (SELECT NOX as value
                 FROM boston
                 ORDER BY NOX
                 LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                 OFFSET (SELECT (COUNT(NOX) - 1) * 4 / 10 FROM BOSTON)
                 )) as value_NOX
    UNION
    SELECT 50 as Percentile,
           (SELECT AVG(value)
           FROM (SELECT INDUS as value
                 FROM boston
                 ORDER BY INDUS
                 LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                 OFFSET (SELECT (COUNT(INDUS) - 1) * 5 / 10 FROM BOSTON)
                 )) as value_INDUS,
           (SELECT AVG(value)
           FROM (SELECT NOX as value
                 FROM boston
                 ORDER BY NOX
                 LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                 OFFSET (SELECT (COUNT(NOX) - 1) * 5 / 10 FROM BOSTON)
                 )) as value_NOX
    UNION
    SELECT 60 as Percentile,
           (SELECT AVG(value)
           FROM (SELECT INDUS as value
                 FROM boston
                 ORDER BY INDUS
                 LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                 OFFSET (SELECT (COUNT(INDUS) - 1) * 6 / 10 FROM BOSTON)
                 )) as value_INDUS,
           (SELECT AVG(value)
           FROM (SELECT NOX as value
                 FROM boston
                 ORDER BY NOX
                 LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                 OFFSET (SELECT (COUNT(NOX) - 1) * 6 / 10 FROM BOSTON)
                 )) as value_NOX
    UNION
    SELECT 70 as Percentile,
           (SELECT AVG(value)
           FROM (SELECT INDUS as value
                 FROM boston
                 ORDER BY INDUS
                 LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                 OFFSET (SELECT (COUNT(INDUS) - 1) * 7 / 10 FROM BOSTON)
                 )) as value_INDUS,
           (SELECT AVG(value)
           FROM (SELECT NOX as value
                 FROM boston
                 ORDER BY NOX
                 LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                 OFFSET (SELECT (COUNT(NOX) - 1) * 7 / 10 FROM BOSTON)
                 )) as value_NOX
    UNION
    SELECT 80 as Percentile,
           (SELECT AVG(value)
           FROM (SELECT INDUS as value
                 FROM boston
                 ORDER BY INDUS
                 LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                 OFFSET (SELECT (COUNT(INDUS) - 1) * 8 / 10 FROM BOSTON)
                 )) as value_INDUS,
           (SELECT AVG(value)
           FROM (SELECT NOX as value
                 FROM boston
                 ORDER BY NOX
                 LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                 OFFSET (SELECT (COUNT(NOX) - 1) * 8 / 10 FROM BOSTON)
                 )) as value_NOX
    UNION
    SELECT 90 as Percentile,
           (SELECT AVG(value)
           FROM (SELECT INDUS as value
                 FROM boston
                 ORDER BY INDUS
                 LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                 OFFSET (SELECT (COUNT(INDUS) - 1) * 9 / 10 FROM BOSTON)
                 )) as value_INDUS,
           (SELECT AVG(value)
           FROM (SELECT NOX as value
                 FROM boston
                 ORDER BY NOX
                 LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                 OFFSET (SELECT (COUNT(NOX) - 1) * 9 / 10 FROM BOSTON)
                 )) as value_NOX
    UNION
    SELECT 100 as Percentile,
           (SELECT AVG(value)
           FROM (SELECT INDUS as value
                 FROM boston
                 ORDER BY INDUS
                 LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
                 OFFSET (SELECT (COUNT(INDUS) - 1) FROM BOSTON)
                 )) as value_INDUS,
           (SELECT AVG(value)
           FROM (SELECT NOX as value
                 FROM boston
                 ORDER BY NOX
                 LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
                 OFFSET (SELECT (COUNT(NOX) - 1) FROM BOSTON)
                 )) as value_NOX
    """,
    con,
)

,Percentile,value_INDUS,value_NOX
0,10,2.91,0.42700
1,20,4.39,0.44245
2,30,5.96,0.47200
3,40,7.38,0.50700
4,50,9.69,0.53800
5,60,12.83,0.57750
6,70,18.10,0.60500
7,80,18.10,0.66950
8,90,19.58,0.71300
9,100,27.74,0.87100
